<a href="https://www.kaggle.com/code/lincolnruwambara/titanic-prediction?scriptVersionId=122026745" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


Importing the data and spliyting it into two different data frames

In [2]:
train =pd.read_csv('/kaggle/input/titanic/train.csv')
test =pd.read_csv('/kaggle/input/titanic/test.csv')
train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


Checking the columns to use for the training data

In [3]:
train.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [4]:
test.columns
test["Survived"] = int

In [5]:
test

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,<class 'int'>
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,<class 'int'>
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,<class 'int'>
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,<class 'int'>
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,<class 'int'>
...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S,<class 'int'>
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C,<class 'int'>
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S,<class 'int'>
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S,<class 'int'>


Selecting the data columns to use for my training and testing

In [6]:
y_train=train.Survived
y_test=test.Survived

customer_info=['Pclass','Sex','Age','Fare']
X_train=train[customer_info]
X_test=test[customer_info]


In [7]:
X_train


,Pclass,Sex,Age,Fare
0,3,male,22.0,7.2500
1,1,female,38.0,71.2833
2,3,female,26.0,7.9250
3,1,female,35.0,53.1000
4,3,male,35.0,8.0500
...,...,...,...,...
886,2,male,27.0,13.0000
887,1,female,19.0,30.0000
888,3,female,NaN,23.4500
889,1,male,26.0,30.0000


In [8]:
X_train.head()

,Pclass,Sex,Age,Fare
0,3,male,22.0,7.2500
1,1,female,38.0,71.2833
2,3,female,26.0,7.9250
3,1,female,35.0,53.1000
4,3,male,35.0,8.0500


Modelling my data to make sure the data is in numeric fomart

In [9]:
X_train_hot=pd.get_dummies(X_train)
X_train_hot 

,Pclass,Age,Fare,Sex_female,Sex_male
0,3,22.0,7.2500,0,1
1,1,38.0,71.2833,1,0
2,3,26.0,7.9250,1,0
3,1,35.0,53.1000,1,0
4,3,35.0,8.0500,0,1
...,...,...,...,...,...
886,2,27.0,13.0000,0,1
887,1,19.0,30.0000,1,0
888,3,NaN,23.4500,1,0
889,1,26.0,30.0000,0,1


In [10]:
X_test_hot=pd.get_dummies(X_test)
X_test_hot 

,Pclass,Age,Fare,Sex_female,Sex_male
0,3,34.5,7.8292,0,1
1,3,47.0,7.0000,1,0
2,2,62.0,9.6875,0,1
3,3,27.0,8.6625,0,1
4,3,22.0,12.2875,1,0
...,...,...,...,...,...
413,3,NaN,8.0500,0,1
414,1,39.0,108.9000,1,0
415,3,38.5,7.2500,0,1
416,3,NaN,8.0500,0,1


Modelling my data to make sure there are no missing fields, used the SimpleImputer() fuction to fill in the missing fields

In [11]:
my_imputer = SimpleImputer()
train_with_imputed_values = my_imputer.fit_transform(X_train_hot)

Fitting my data in preperation for my modell

In [12]:

test_with_imputed_values = my_imputer.fit_transform(X_test_hot)
test_with_imputed_values

array([[ 3.        , 34.5       ,  7.8292    ,  0.        ,  1.        ],
       [ 3.        , 47.        ,  7.        ,  1.        ,  0.        ],
       [ 2.        , 62.        ,  9.6875    ,  0.        ,  1.        ],
       ...,
       [ 3.        , 38.5       ,  7.25      ,  0.        ,  1.        ],
       [ 3.        , 30.27259036,  8.05      ,  0.        ,  1.        ],
       [ 3.        , 30.27259036, 22.3583    ,  0.        ,  1.        ]])

In [13]:
titanic_model = RandomForestRegressor()

In [14]:
titanic_model.fit(train_with_imputed_values, y_train)

y_pred = titanic_model.predict(test_with_imputed_values)



In [15]:
def score(res):
    return round(res * 100, 2)

In [16]:
score_rf = score(titanic_model.score(train_with_imputed_values, y_train))
score_rf

87.84